<a href="https://colab.research.google.com/github/alexandr-khvatov/text_classification/blob/main/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аугментация данных


In [2]:
!pip install transformers==4.1.1 sentencepiece==0.1.94
!pip install mosestokenizer==1.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 37.4 MB/s 
     |████████████████████████████████| 1.1 MB 63.0 MB/s 
     |████████████████████████████████| 880 kB 67.6 MB/s 
     |████████████████████████████████| 2.9 MB 67.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=9313557b5f77da3f017448a22c57f1c7967ec8195079c8e2f76d41e4a823116b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49118 sha256=3967cc1c16f9b61e9d8340ee8228c45a6e4a868030baf0ef8ed8cb5f0242170c
  Stored in directory: /root/.cache/pip/wheels/9f/6e/ed/6e051d73e060472bb12a1cda32ba756d05fd7597191ee6ef9c


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

target_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

target_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-ru")

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/307M [00:00<?, ?B/s]

Some weights of MarianMTModel were not initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ru and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
def translate(text, model, tokenizer, language="en"):
    # Prepare the text data into appropriate format for the model
    template = lambda text: f"{text}" if language == "en" else f">>{language}<< {text}"
    src_texts =[template(text)] 

    # Tokenize the texts
    encoded = tokenizer.prepare_seq2seq_batch(src_texts,
                                              return_tensors='pt')
    
    # Generate translation using model
    translated = model.generate(**encoded)

    # Convert the generated tokens indices back into text
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    
    return translated_texts

In [5]:
def back_translate(text, source_lang="ru", target_lang="en"):
    # Translate to target language
    en_texts = translate(text, target_model, target_tokenizer, 
                         language=target_lang)

    # Translate from target language back to source language
    back_translated_texts = translate(en_texts, en_model, en_tokenizer, 
                                      language=source_lang)
    
    return back_translated_texts

### Perform Augmentation using English <-> Spanish


In [8]:
def make_back_translate(text):
  return back_translate(text, source_lang="ru", target_lang="en")[0]

In [10]:
aug_texts = make_back_translate('Зачем ты пишешь хуйню, дегенерат? Поцелуй в губы ! поцелую в засос.')
print(aug_texts)

Почему ты пишешь дерьмо, дегенерат?


In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('https://raw.githubusercontent.com/alexandr-khvatov/text_classification/main/data/data_original_RU.csv')
df.head(10)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
5,Для каких стан является эталоном современная с...,1.0
6,В шапке были ссылки на инфу по текущему фильму...,0.0
7,УПАД Т! ТАМ НЕЛЬЗЯ СТРОИТЬ! ТЕХНОЛОГИЙ НЕТ! РА...,1.0
8,"Ебать тебя разносит, шизик.\n",1.0
9,"Обосрался, сиди обтекай\n",1.0


In [ ]:
%%time
df['comment_backtranslate']=df['comment'].apply(make_back_translate)

In [ ]:
print('ок')